In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import time

# 2次元形態係数を計算するモジュール

2次元空間の位置ベクトルから、その座標で形成される閉空間の任意の2面間の形態係数を計算する。

### 入力値  

$ \boldsymbol{q}_{i} $ ：点$i$の位置ベクトル

### 出力値
$ F_{ij} $ ：面$i$から面$j$をみる形態係数

In [2]:
class tvf_input():
    def __init__( self, q ):
        self.q = q

# q：点の位置ベクトル

### 1. 2面間の形態係数の計算

2次元空間が仮定できる場合、Hottel's Crossed-String Methodが適用できる。

（参考：Hottel,H.C.: Radiant Heat Transmission, in W. H. McAdams(ed.), Heat Transmission, 3rd ed., McGraw-Hill, New York, 1954.）  

そのため、面$i$から面$j$をみる形態係数$F_{ij}$は、

$$
\begin{eqnarray}
 &\displaystyle F_{ij} = \max \biggl(0, \frac{L_{1} + L_{2} - L_{0} - L_{3}}{2 \Delta L_{i}}\biggr)& \qquad\qquad\text{(1)} \nonumber\\
\end{eqnarray}
$$

$$
\begin{eqnarray}
 &\displaystyle L_{0} = \bigl|\boldsymbol{D} - \boldsymbol{A}\bigr|& \qquad\qquad\text{(2)} \nonumber\\
\end{eqnarray}
$$

$$
\begin{eqnarray}
 &\displaystyle L_{1} = \bigl|\boldsymbol{C} - \boldsymbol{A}\bigr|& \qquad\qquad\text{(3)} \nonumber\\
\end{eqnarray}
$$

$$
\begin{eqnarray}
 &\displaystyle L_{2} = \bigl|\boldsymbol{D} - \boldsymbol{B}\bigr|& \qquad\qquad\text{(4)} \nonumber\\
\end{eqnarray}
$$

$$
\begin{eqnarray}
 &\displaystyle L_{3} = \bigl|\boldsymbol{C} - \boldsymbol{B}\bigr|& \qquad\qquad\text{(5)} \nonumber\\
\end{eqnarray}
$$

![image5.png][attached1]
[attached1]:img\image5.png  

In [3]:
def hottels_crossed_string_method(A, B, C, D, dLi):
    
    L = np.zeros(4)
    
    L[0] = ((D[0] - A[0]) ** 2. + (D[1] - A[1]) ** 2.) ** 0.5      #式(2)、ij間の長さ1(m)
    L[1] = ((C[0] - A[0]) ** 2. + (C[1] - A[1]) ** 2.) ** 0.5      #式(3)、ij間の長さ2(m)
    L[2] = ((D[0] - B[0]) ** 2. + (D[1] - B[1]) ** 2.) ** 0.5      #式(4)、ij間の長さ3(m)
    L[3] = ((C[0] - B[0]) ** 2. + (C[1] - B[1]) ** 2.) ** 0.5      #式(5)、ij間の長さ4(m)
    F = (L[1] + L[2] - L[0] - L[3]) / (2. * dLi)                   #式(6)、形態係数
    
    return F

#### ある面からみる点の形態係数  

二次元空間が仮定できる場合、面$i$から点$j$をみる形態係数$f_{ij}$は、下記のように表すことができる。

（参考：デザイナーのための建築環境計画 熱・日射・光・風, 猪岡 達夫, 丸善出版, 2015）

$$
\begin{eqnarray}
 &\displaystyle L_{1} &= \bigl|\boldsymbol{C} - \boldsymbol{A}\bigr| = \bigl|\boldsymbol{C} - \boldsymbol{B}\bigr| = \Delta L_{i}& \qquad\qquad\text{(6)} \nonumber\\
\end{eqnarray}
$$

$$
\begin{eqnarray}
 &\displaystyle L_{3} &= \bigl|\boldsymbol{C} - \boldsymbol{B}\bigr| = \bigl|\boldsymbol{B} - \boldsymbol{B}\bigr| = 0& \qquad\qquad\text{(7)} \nonumber\\
\end{eqnarray}
$$

したがって、

$$
\begin{eqnarray}
 &\displaystyle f_{ij} = \max \biggl(0, \frac{\Delta L_{i} + L_{2} - L_{0}}{2 \Delta L_{i}}\biggr)& \qquad\qquad\text{(8)} \nonumber\\
\end{eqnarray}
$$

![image6.png][attached1]
[attached1]:img\image6.png  

#### 点A～点Bの距離  

三平方の定理より、

$$
\begin{eqnarray}
 &\displaystyle L = \sqrt{(A_x - B_x)^2 + (A_y - B_y)^2}& \qquad\qquad\text{(6)} \nonumber\\
\end{eqnarray}
$$

In [4]:
def distance_calc(A, B):
    return ((A[0] - B[0]) ** 2. + (A[1] - B[1]) ** 2.) ** 0.5

#### 同じ形態係数となる点の探索

In [5]:
def cross_point_optimize(x, qa, va, qb, qc, pf, dL): #点jの形態係数と同じ形態係数になる点k'の探索
    return ((qa[0] + x * va[0] - qb[0]) ** 2. + (qa[1] + x * va[1] - qb[1]) ** 2.) ** 0.5 \
           - ((qa[0] + x * va[0] - qc[0]) ** 2. + (qa[1] + x * va[1] - qc[1]) ** 2.) ** 0.5 \
           - (2. * pf - 1.) * dL

In [6]:
def two_dimensional_view_factor(L):
        
    v = np.array([L.q[i+1] - L.q[i] for i in range(len(L.q) - 1)], dtype=float)                    #方向ベクトルv[i]
    ve= np.array([v[i] / np.linalg.norm(v[i]) for i in range(len(L.q) - 1)], dtype=float)          #単位ベクトルve[i]
    u = np.array([[- v[i][1], v[i][0]] for i in range(len(L.q) - 1)], dtype=float)                 #法線ベクトルu[i]
    
    w3d = np.array([[np.append(L.q[j] - L.q[i], 0.) \
            for j in range(len(L.q))] for i in range(len(L.q))], dtype=float)                      #点iから点jへの方向ベクトルw3d[i][j]の計算
    
    dLi = np.array([distance_calc(L.q[i+1], L.q[i]) for i in range(len(L.q) - 1)], dtype=float)    #領域境界分割要素の長さ(mm)
    
    pf = np.array([[hottels_crossed_string_method(L.q[i], L.q[i+1], L.q[i+1], L.q[j], dLi[i]) \
            for j in range(len(L.q))] for i in range(len(L.q) - 1)], dtype=float)                  #点の形態係数
    
    cross_w3d = np.array([[np.cross(w3d[i][i+1], w3d[i][j]) \
            for j in range(len(L.q))] for i in range(len(L.q) - 1)], dtype=float)                  #v[i]とw[i][j]の外積
    
    qdj1 = np.array([[L.q[j] for j in range(len(L.q))] for i in range(len(L.q) - 1)], dtype=float) #q[j]が負、q[j+1]が正
    qdj2 = np.array([[L.q[j] for j in range(len(L.q))] for i in range(len(L.q) - 1)], dtype=float) #q[j]が正、q[j+1]が負
    
    #7.q[i]+ai[i][j]*v[i]=q[j]+aj[i][j]*v[j]となるai[i][j]、aj[i][j]の計算
    ai = np.zeros((len(L.q) - 1, len(L.q) - 1))
    aj = np.zeros((len(L.q) - 1, len(L.q) - 1))
    qm = np.empty((len(L.q) - 1, len(L.q) - 1, 2), dtype=object)
    for i in range(len(L.q) - 1):
        for j in range(len(L.q) - 1):           
            if (ve[i] == ve[j]).all() or (ve[i] == -ve[j]).all(): #平行なベクトル
                ai[i][j] = None
                aj[i][j] = None
            else:
                ai[i][j] = np.linalg.solve(np.array([v[i], -v[j]]).T.astype(np.float64), (L.q[j] - L.q[i]).astype(np.float64))[0]
                aj[i][j] = np.linalg.solve(np.array([v[i], -v[j]]).T.astype(np.float64), (L.q[j] - L.q[i]).astype(np.float64))[1]
            
            #8.qdj1[i][j]、qdj2[i][j]、vdj1[i][j]、vdj2[i][j]、pf[i][j]の計算、
            #  q[j](またはqdj1[i][j])とq[i][j+1](またはqdj2[i][j+1])の中点の位置ベクトルqm[i][j]の計算
            
            if (0. < aj[i][j] and aj[i][j] < 1.) and (cross_w3d[i][j][2] < 0. and cross_w3d[i][j+1][2] > 0.): #q[j]が負、q[j+1]が正
                qdj1[i][j] = L.q[j] + aj[i][j] * v[j] #点j'の位置ベクトル
                pf[i][j] = np.round(hottels_crossed_string_method(L.q[i], L.q[i+1], L.q[i+1], qdj1[i][j], dLi[i]))   #点の形態係数
                qm[i][j] = (L.q[j+1] + qdj1[i][j]) / 2.
            elif (0. < aj[i][j] and aj[i][j] < 1.) and (cross_w3d[i][j][2] > 0. and cross_w3d[i][j+1][2] < 0.): #q[j]が正、q[j+1]が負 
                qdj2[i][j+1] = L.q[j] + aj[i][j] * v[j] #点j'の位置ベクトル
                pf[i][j+1] = np.round(hottels_crossed_string_method(L.q[i], L.q[i+1], L.q[i+1], qdj2[i][j+1], dLi[i]))   #点の形態係数
                qm[i][j] = (qdj2[i][j+1] + L.q[j]) / 2.
            else:
                qm[i][j] = (L.q[j+1] + L.q[j]) / 2.
            
    um = np.array([[qm[i][j] - qm[i][i] for j in range(len(L.q) - 1)] for i in range(len(L.q) - 1)], dtype=float) #qm[i][i]からqm[i][j]への方向ベクトルum
    
    dot_u = np.array([[np.vdot(u[i], um[i][j])  for j in range(len(L.q) - 1)] for i in range(len(L.q) - 1)], dtype=float) #u[i]とum[i][j]の内積dot_u
                                        
    #面i→面jの形態係数F[i][j]の計算
    F = np.zeros((len(L.q) - 1, len(L.q) - 1))   #面の形態係数
    for i in range(len(L.q) - 1):
        for j in range(len(L.q) - 1):
            if dot_u[i][j] > 0. and dot_u[j][i] > 0.:
                F[i][j] = pf[i][j+1] - pf[i][j]
                
   #重複部分の判断
    
    qdk1 = np.array([[L.q[k] for k in range(len(L.q))] for i in range(len(L.q) - 1)], dtype=float)
    qdk2 = np.array([[L.q[k] for k in range(len(L.q))] for i in range(len(L.q) - 1)], dtype=float) 

    j1 = np.array([[k for k in range(len(L.q) - 1)] for i in range(len(L.q) - 1)], dtype=int)
    j2 = np.array([[k+1 for k in range(len(L.q) - 1)] for i in range(len(L.q) - 1)], dtype=int)
            
    for i in range(len(L.q) - 1):     
        seej1 = np.ones(len(L.q) - 1)    #点jがみえるかの判断初期値(1:見える、0:見えない)
        seej2 = np.ones(len(L.q) - 1)    #点jがみえるかの判断初期値(1:見える、0:見えない)
        seejh1 = np.ones(len(L.q) - 1)    #点jがみえるかの判断初期値(1:見える、0:見えない)
        seejh2 = np.ones(len(L.q) - 1)    #点jがみえるかの判断初期値(1:見える、0:見えない)
        for j in range(len(L.q) - 1):

            disj = distance_calc(L.q[j], L.q[i+1]) + distance_calc(L.q[j], L.q[i])                    #面iの端点と点jの距離
            disj1 = distance_calc(qdj1[i][j], L.q[i+1]) + distance_calc(qdj1[i][j], L.q[i])       #面iの端点と点j'の距離
            disj2 = distance_calc(qdj2[i][j+1], L.q[i+1]) + distance_calc(qdj2[i][j+1], L.q[i])   #面iの端点と点j'の距離
            ak1 = np.ones(len(L.q) - 1)                                                #点k1の位置を特定する係数の初期値
            ak2 = np.ones(len(L.q) - 1)                                                #点k2の位置を特定する係数の初期値
            disk1 = np.zeros(len(L.q) - 1)                                             #面iの端点と点k1の距離の初期値
            disk2 = np.zeros(len(L.q) - 1)                                             #面iの端点と点k2の距離の初期値
            diskh1 = np.zeros(len(L.q) - 1)                                             #面iの端点と点k1の距離の初期値
            diskh2 = np.zeros(len(L.q) - 1)                                             #面iの端点と点k2の距離の初期値
            
            if dot_u[i][j] > 0. and dot_u[j][i] > 0.:
                if pf[i][j] == 0.:
                    for k in range(len(L.q) - 1):
                        if dot_u[i][k] > 0. and dot_u[k][i] > 0. and pf[i][k] == 0.:
                            diskh1[k] = distance_calc(qdj1[i][k], L.q[i+1]) + distance_calc(qdj1[i][k], L.q[i])
                            if j != k and disj1 >= diskh1[k]:
                                seejh1[j] = 0
                        
                elif pf[i][j+1] == 1.:
                    for k in range(len(L.q) - 1):
                        if dot_u[i][k] > 0. and dot_u[k][i] > 0. and pf[i][k+1] == 1.:
                            diskh2[k] = distance_calc(qdj2[i][k+1], L.q[i+1]) + distance_calc(qdj2[i][k+1], L.q[i])
                            if j != k and disj2 >= diskh2[k]:
                                seejh2[j] = 0
                    
            if j != i:
                if (j == 0 and dot_u[i][len(L.q) - 2] > 0. and dot_u[len(L.q) - 2][i] > 0.) or \
                   (j != 0 and dot_u[i][j-1] > 0. and dot_u[j-1][i] > 0.):
                    if (j == 0 and pf[i][len(L.q) - 2] < pf[i][j] and pf[i][j] >= pf[i][j+1]) or \
                       (j != 0 and pf[i][j-1] < pf[i][j] and pf[i][j] >= pf[i][j+1]):
                        if not (dot_u[j-1][j] > 0. and dot_u[j][j-1] > 0.):
                            for k in range(len(L.q) - 1):
                                if j != k and (L.q[j] != L.q[k]).all():
                                    if dot_u[i][k] > 0. and dot_u[k][i] > 0.:
                                        if pf[i][k] <= pf[i][j] and pf[i][j] < pf[i][k+1]:
                                            ak1[k] = optimize.fsolve(cross_point_optimize, 0, args=(L.q[k], v[k], L.q[i+1], L.q[i], pf[i][j], dLi[i]))
                                            if 0. <= ak1[k] and ak1[k] < 1.:
                                                qdk1[i][k] = L.q[k] + ak1[k] * v[k]
                                                disk1[k] = distance_calc(qdk1[i][k], L.q[i+1]) + distance_calc(qdk1[i][k], L.q[i])
                                                if disj >= disk1[k]:
                                                    seej1[j] = 0

                        for k in range(len(L.q) - 1):
                            if 0. <= ak1[k] and ak1[k] < 1.:
                                if j != k and seej1[j] == 1 and disj < disk1[k]:
                                    j1[i][k] = j
                                    #点j～点kまでの形態係数Fの処理
                                    if j <= k:
                                        for kk in range(j,k):
                                            F[i][kk] = 0.
                                    else: #点0をまたぐとき
                                        for kk in range(k):
                                            F[i][kk] = 0.
                                        for kk in range(j,len(L.q) - 1):
                                            F[i][kk] = 0. 
                    
            if j != i+1:
                if dot_u[i][j] > 0. and dot_u[j][i] > 0.:
                    if (j == 0 and pf[i][len(L.q) - 2] >= pf[i][j] and pf[i][j] < pf[i][j+1]) or \
                       (j != 0 and pf[i][j-1] >= pf[i][j] and pf[i][j] < pf[i][j+1]):
                        if not (dot_u[j-1][j] > 0. and dot_u[j][j-1] > 0.):
                            for k in range(len(L.q) - 1):
                                if j != k and (L.q[j] != L.q[k]).all():
                                    if dot_u[i][k] > 0. and dot_u[k][i] > 0.:            
                                        if pf[i][k] <= pf[i][j] and pf[i][j] < pf[i][k+1]:
                                            ak2[k] = optimize.fsolve(cross_point_optimize, 0, args=(L.q[k], v[k], L.q[i+1], L.q[i], pf[i][j], dLi[i]))
                                            if 0. <= ak2[k] and ak2[k] < 1.:
                                                qdk2[i][k] = L.q[k] + ak2[k] * v[k]
                                                disk2[k] = distance_calc(qdk2[i][k], L.q[i+1]) + distance_calc(qdk2[i][k], L.q[i])
                                                if disj >= disk2[k]:
                                                    seej2[j] = 0

                        for k in range(len(L.q) - 1):
                            if j != k and 0. <= ak2[k] and ak2[k] < 1.:
                                if seej2[j] == 1 and disj < disk2[k]:
                                    j2[i][k] = j
                                    #点k+1～点j-1までの形態係数Fの処理
                                    if k+1 <= j:
                                        for kk in range(k+1,j):
                                            F[i][kk] = 0. #ここに問題あり(i=4の事案)
                                    else: #点0をまたぐとき
                                        for kk in range(j):
                                            F[i][kk] = 0.
                                        for kk in range(k+1, len(L.q) - 1):
                                            F[i][kk] = 0.
            
            
        for j in range(len(L.q) - 1):            
            if dot_u[i][j] > 0. and dot_u[j][i] > 0.:
                if pf[i][j] == 0. and seejh1[j] == 1:
                    #面i+1～面j-1までの形態係数Fの処理
                    if i < j:
                        if i+1 != j:
                            for jj in range(i+1,j):
                                F[i][jj] = 0.
                    elif i > j: #点0をまたぐとき
                        if j != 0:
                            for jj in range(j):
                                F[i][jj] = 0.
                        if i+1 != len(L.q) - 1:
                            for jj in range(i+1, len(L.q) - 1):
                                F[i][jj] = 0.
                                
                                
                elif pf[i][j+1] == 1. and seejh2[j] == 1:
                    #面j+1～点i-1までの形態係数Fの処理
                    if i > j:
                        if i != j+1:
                            for jj in range(j+1,i):
                                F[i][jj] = 0.
                    elif i < j:
                        if i != 0:
                            for jj in range(i):
                                F[i][jj] = 0.
                        if j+1 != len(L.q) - 1: 
                            for jj in range(j+1, len(L.q) - 1):
                                F[i][jj] = 0.

        for k in range(len(L.q) - 1):
            if i != k and (j1[i][k] != k or j2[i][k] != k+1):
                F[i][k] = pf[i][j2[i][k]] - pf[i][j1[i][k]]
       
    return F

# 実行ファイル

In [7]:
if __name__ == '__main__':
    
    tst = time.time()
    '''
    #75
    tvf_in = tvf_input(np.array([[0.0, 0.0],
                                [0.04330764779948737, 1.7525950675481146],
                                [0.2560599548476148, 4.255178343090272],
                                [0.6453044689501155, 6.7364432026767584],
                                [1.2090910776108572, 9.18395852542194],
                                [1.9445952135697624, 11.585462275401303],
                                [2.8481320058665105, 13.928922934372075],
                                [3.91517474102017, 16.202599779601723],
                                [5.140377541834432, 18.39510170481285],
                                [6.517602150207807, 20.49544428955193],
                                [6.517602150207807, 22.64544428955193],
                                [6.517602150207807, 24.79544428955193],
                                [6.517602150207807, 26.94544428955193],
                                [6.517602150207807, 29.09544428955193],
                                [6.517602150207807, 31.24544428955193],
                                [6.517602150207807, 33.39544428955193],
                                [6.517602150207807, 35.54544428955193],
                                [6.517602150207807, 37.695444289551936],
                                [6.517602150207807, 39.845444289551935],
                                [6.517602150207801, 41.995444289551926],
                                [5.140377541834426, 39.895101704812845],
                                [3.915174741020164, 37.70259977960171],
                                [2.8481320058665043, 35.42892293437207],
                                [1.9445952135697562, 33.0854622754013],
                                [1.209091077610851, 30.683958525421925],
                                [0.6453044689501093, 28.23644320267675],
                                [0.2560599548476086, 25.75517834309026],
                                [0.04330764779948115, 23.252595067548103],
                                [0.008113435486920118, 20.741231302061845],
                                [0.150653640685797, 18.23366896284085],
                                [0.008113435486920118, 20.741231302061845],
                                [0.0, 21.5],
                                [0.0, 19.35],
                                [0.0, 17.2],
                                [0.0, 15.05],
                                [0.0, 12.900000000000002],
                                [0.0, 10.75],
                                [0.0, 8.6],
                                [0.0, 6.450000000000001],
                                [0.0, 4.300000000000001],
                                [0.0, 2.149999999999999],
                                [0.0, 4.440892098500626e-16]]))
    '''
    '''
    #45
    tvf_in = tvf_input(np.array([[0.0, 1.7763568394002505e-15],
                                [1.130337730847625, 2.242882789937955],
                                [2.4164986955196177, 4.400190502836519],
                                [3.852039235908123, 6.4611150410180365],
                                [5.429767303197853, 8.415331185886313],
                                [7.141778489984123, 10.253048327204937],
                                [8.979495631302743, 11.9650595139912],
                                [10.93371177617102, 13.542787581280937],
                                [12.994636314352537, 14.978328121669445],
                                [15.151944027251098, 16.264489086341438],
                                [17.394826817189056, 17.394826817189063],
                                [17.394826817189056, 19.544826817189062],
                                [17.394826817189056, 21.694826817189064],
                                [17.394826817189056, 23.844826817189062],
                                [17.394826817189056, 25.994826817189065],
                                [17.394826817189056, 28.144826817189063],
                                [17.394826817189056, 30.294826817189062],
                                [17.394826817189056, 32.44482681718906],
                                [17.394826817189056, 34.59482681718906],
                                [17.394826817189056, 36.744826817189065],
                                [17.394826817189042, 38.894826817189056],
                                [15.151944027251083, 37.76448908634143],
                                [12.994636314352523, 36.478328121669435],
                                [10.933711776171005, 35.04278758128093],
                                [8.97949563130273, 33.46505951399119],
                                [7.141778489984109, 31.75304832720493],
                                [5.429767303197838, 29.915331185886302],
                                [3.8520392359081086, 27.961115041018033],
                                [2.4164986955196035, 25.90019050283651],
                                [1.1303377308476108, 23.74288278993795],
                                [-1.4210854715202004e-14, 21.499999999999993],
                                [-1.4210854715202004e-14, 19.349999999999994],
                                [-1.4210854715202004e-14, 17.199999999999992],
                                [-1.4210854715202004e-14, 15.049999999999994],
                                [-1.4210854715202004e-14, 12.899999999999993],
                                [-1.4210854715202004e-14, 10.749999999999993],
                                [-1.4210854715202004e-14, 8.599999999999993],
                                [-1.4210854715202004e-14, 6.449999999999994],
                                [-1.4210854715202004e-14, 4.299999999999994],
                                [-1.4210854715202004e-14, 2.1499999999999915],
                                [-1.4210854715202004e-14, -7.105427357601002e-15]]))
    '''
    '''
    #0
    tvf_in = tvf_input(np.array([[1.7763568394002505e-15, -7.105427357601002e-15],
                                [2.385227104685109, 0.7866881556583536],
                                [4.82012715739872, 1.4026819287379766],
                                [7.292501324637542, 1.8448951944229677],
                                [9.789963027827046, 2.111112467114637],
                                [12.300000000000006, 2.1999999999999957],
                                [14.81003697217296, 2.111112467114637],
                                [17.30749867536247, 1.8448951944229677],
                                [19.779872842601293, 1.4026819287379766],
                                [22.214772895314898, 0.7866881556583536],
                                [24.6, -7.105427357601002e-15],
                                [24.6, 2.149999999999993],
                                [24.6, 4.299999999999993],
                                [24.6, 6.449999999999993],
                                [24.6, 8.599999999999993],
                                [24.6, 10.749999999999993],
                                [24.6, 12.899999999999993],
                                [24.6, 15.049999999999992],
                                [24.6, 17.199999999999992],
                                [24.6, 19.349999999999994],
                                [24.60000000000001, 21.499999999999986],
                                [22.2147728953149, 22.286688155658346],
                                [19.7798728426013, 22.90268192873797],
                                [17.307498675362474, 23.34489519442296],
                                [14.81003697217296, 23.61111246711463],
                                [12.300000000000011, 23.69999999999999],
                                [9.78996302782705, 23.61111246711463],
                                [7.292501324637545, 23.34489519442296],
                                [4.820127157398723, 22.90268192873797],
                                [2.3852271046851143, 22.286688155658346],
                                [7.105427357601002e-15, 21.499999999999986],
                                [7.105427357601002e-15, 19.349999999999987],
                                [7.105427357601002e-15, 17.199999999999985],
                                [7.105427357601002e-15, 15.049999999999986],
                                [7.105427357601002e-15, 12.899999999999986],
                                [7.105427357601002e-15, 10.749999999999986],
                                [7.105427357601002e-15, 8.599999999999985],
                                [7.105427357601002e-15, 6.449999999999987],
                                [7.105427357601002e-15, 4.2999999999999865],
                                [7.105427357601002e-15, 2.1499999999999844],
                                [7.105427357601002e-15, -1.4210854715202004e-14]]))
    '''
    '''
    #-45
    tvf_in = tvf_input(np.array([[1.7763568394002505e-15, 0.0],
                                [2.2428827899379584, -1.130337730847625],
                                [4.400190502836516, -2.4164986955196177],
                                [6.4611150410180365, -3.852039235908123],
                                [8.415331185886313, -5.429767303197853],
                                [10.253048327204937, -7.141778489984123],
                                [11.9650595139912, -8.979495631302743],
                                [13.542787581280937, -10.933711776171016],
                                [14.978328121669445, -12.994636314352537],
                                [16.264489086341438, -15.151944027251098],
                                [17.394826817189063, -17.394826817189053],
                                [17.394826817189063, -15.244826817189052],
                                [17.394826817189063, -13.094826817189052],
                                [17.394826817189063, -10.944826817189053],
                                [17.394826817189063, -8.794826817189053],
                                [17.394826817189063, -6.6448268171890525],
                                [17.394826817189063, -4.494826817189052],
                                [17.394826817189063, -2.3448268171890536],
                                [17.394826817189063, -0.19482681718905326],
                                [17.394826817189063, 1.9551731828109489],
                                [17.394826817189056, 4.105173182810965],
                                [16.264489086341428, 6.34805597274892],
                                [14.978328121669435, 8.505363685647481],
                                [13.54278758128093, 10.566288223829002],
                                [11.965059513991193, 12.520504368697274],
                                [10.25304832720493, 14.358221510015895],
                                [8.415331185886306, 16.070232696802165],
                                [6.461115041018029, 17.647960764091895],
                                [4.400190502836509, 19.0835013044804],
                                [2.2428827899379513, 20.369662269152393],
                                [-7.105427357601002e-15, 21.500000000000018],
                                [-7.105427357601002e-15, 19.35000000000002],
                                [-7.105427357601002e-15, 17.200000000000017],
                                [-7.105427357601002e-15, 15.050000000000018],
                                [-7.105427357601002e-15, 12.900000000000018],
                                [-7.105427357601002e-15, 10.750000000000018],
                                [-7.105427357601002e-15, 8.600000000000017],
                                [-7.105427357601002e-15, 6.450000000000019],
                                [-7.105427357601002e-15, 4.3000000000000185],
                                [-7.105427357601002e-15, 2.1500000000000163],
                                [-7.105427357601002e-15, 1.7763568394002505e-14]]))
    '''    
    #'''
    #-75
    tvf_in = tvf_input(np.array([[0.0, 3.552713678800501e-15],
                                [1.3772246083733783, -2.1003425847390815],
                                [2.6024274091876407, -4.292844509950207],
                                [3.6694701443412967, -6.566521355179852],
                                [4.573006936638045, -8.909982014150625],
                                [5.308511072596957, -11.31148576412999],
                                [5.872297681257699, -13.75900108687517],
                                [6.2615421953601995, -16.240265946461655],
                                [6.474294502408327, -18.742849222003816],
                                [6.517602150207814, -20.495444289551937],
                                [6.517602150207814, -18.34544428955194],
                                [6.517602150207814, -16.195444289551936],
                                [6.517602150207814, -14.045444289551938],
                                [6.517602150207814, -11.895444289551937],
                                [6.517602150207814, -9.745444289551937],
                                [6.517602150207814, -7.595444289551937],
                                [6.517602150207814, -5.445444289551938],
                                [6.517602150207814, -3.295444289551938],
                                [6.517602150207814, -1.1454442895519357],
                                [6.517602150207814, 1.004555710448063],
                                [6.509488714720889, 0.24578701250992907],
                                [6.366948509522012, -2.2617753267110743],
                                [6.509488714720889, 0.24578701250992907],
                                [6.474294502408328, 2.757150777996184],
                                [6.2615421953602, 5.259734053538345],
                                [5.8722976812577, 7.74099891312483],
                                [5.308511072596958, 10.18851423587001],
                                [4.573006936638046, 12.590017985849375],
                                [3.6694701443412976, 14.933478644820148],
                                [2.6024274091876416, 17.207155490049793],
                                [1.3772246083733792, 19.39965741526092],
                                [8.881784197001252e-16, 21.500000000000004],
                                [8.881784197001252e-16, 19.350000000000005],
                                [8.881784197001252e-16, 17.200000000000003],
                                [8.881784197001252e-16, 15.050000000000004],
                                [8.881784197001252e-16, 12.900000000000004],
                                [8.881784197001252e-16, 10.750000000000004],
                                [8.881784197001252e-16, 8.600000000000003],
                                [8.881784197001252e-16, 6.450000000000005],
                                [8.881784197001252e-16, 4.300000000000004],
                                [8.881784197001252e-16, 2.150000000000002],
                                [8.881784197001252e-16, 3.552713678800501e-15]]))
    #'''     
    
    F = two_dimensional_view_factor(tvf_in)               #面の形態係数
    
    #形態係数と総和の確認
    sumF = np.zeros(len(F))
    for i in range(len(F)):
        for j in range(len(F.T)):
            sumF[i] += F[i][j]
            print(i, j, F[i][j], sumF[i])
            
    tsp =time.time()
    
    print(tsp-tst)
            

0 0 0.0 0.0
0 1 0.0 0.0
0 2 0.0 0.0
0 3 0.0 0.0
0 4 0.0 0.0
0 5 0.0 0.0
0 6 0.0 0.0
0 7 0.0 0.0
0 8 0.0 0.0
0 9 0.0 0.0
0 10 0.0 0.0
0 11 0.0 0.0
0 12 0.0 0.0
0 13 2.6262813090813852e-05 2.6262813090813852e-05
0 14 0.00549203553250441 0.005518298345595224
0 15 0.024644290993948454 0.030162589339543678
0 16 0.06653322285888388 0.09669581219842756
0 17 0.05205252086149735 0.1487483330599249
0 18 0.0 0.1487483330599249
0 19 0.0 0.1487483330599249
0 20 0.0 0.1487483330599249
0 21 0.17784940751782197 0.3265977405777469
0 22 0.1742082044643401 0.500805945042087
0 23 0.12974632747406667 0.6305522725161536
0 24 0.08902724151442332 0.719579514030577
0 25 0.06188210351772894 0.7814616175483059
0 26 0.04485129839747737 0.8263129159457833
0 27 0.03394574064725964 0.8602586565930429
0 28 0.02664319909022006 0.886901855683263
0 29 0.021505026905228486 0.9084068825884914
0 30 0.017712792228395058 0.9261196748168865
0 31 0.0007995786893710832 0.9269192535062576
0 32 0.000977106306440656 0.927896359812

18 0 0.0 0.0
18 1 0.0 0.0
18 2 0.0 0.0
18 3 0.0 0.0
18 4 0.0 0.0
18 5 0.0009877989424148526 0.0009877989424148526
18 6 0.0004951352998910918 0.0014829342423059444
18 7 6.160916337594191e-05 0.0015445434056818863
18 8 1.3493007938736312e-06 0.00154589270647576
18 9 0.0 0.00154589270647576
18 10 0.0 0.00154589270647576
18 11 0.0 0.00154589270647576
18 12 0.0 0.00154589270647576
18 13 0.0 0.00154589270647576
18 14 0.0 0.00154589270647576
18 15 0.0 0.00154589270647576
18 16 0.0 0.00154589270647576
18 17 0.0 0.00154589270647576
18 18 0.0 0.00154589270647576
18 19 0.3529202592515933 0.3544661519580691
18 20 0.6455338480419309 1.0
18 21 0.0 1.0
18 22 0.0 1.0
18 23 0.0 1.0
18 24 0.0 1.0
18 25 0.0 1.0
18 26 0.0 1.0
18 27 0.0 1.0
18 28 0.0 1.0
18 29 0.0 1.0
18 30 0.0 1.0
18 31 0.0 1.0
18 32 0.0 1.0
18 33 0.0 1.0
18 34 0.0 1.0
18 35 0.0 1.0
18 36 0.0 1.0
18 37 0.0 1.0
18 38 0.0 1.0
18 39 0.0 1.0
18 40 0.0 1.0
19 0 0.0 0.0
19 1 0.0 0.0
19 2 0.0 0.0
19 3 0.0 0.0
19 4 0.0 0.0
19 5 0.0 0.0
19 6 0.0 0

33 10 0.00048265561417297313 0.009876534551771465
33 11 0.0014104244263072625 0.011286958978078727
33 12 0.001731178458730026 0.013018137436808753
33 13 0.002155387790743979 0.015173525227552732
33 14 0.002727474860690039 0.01790100008824277
33 15 0.0035162354093907396 0.02141723549763351
33 16 0.004631399674867833 0.026048635172501344
33 17 0.0015539606872477171 0.02760259585974906
33 18 0.0 0.02760259585974906
33 19 0.0 0.02760259585974906
33 20 0.0 0.02760259585974906
33 21 0.00989230837397027 0.03749490423371933
33 22 0.012729144927963183 0.050224049161682514
33 23 0.016962941015676407 0.06718699017735892
33 24 0.024024108728702545 0.09121109890606147
33 25 0.03758693510773503 0.1287980340137965
33 26 0.068706971116402 0.1975050051301985
33 27 0.15326267318023287 0.3507676783104314
33 28 0.33011412954170255 0.6808818078521339
33 29 0.2766183324360514 0.9575001402881853
33 30 0.04249985971181469 1.0
33 31 0.0 1.0
33 32 0.0 1.0
33 33 0.0 1.0
33 34 0.0 1.0
33 35 0.0 1.0
33 36 0.0 1.0
